# SETTINGS

## Importamos Librerías

In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.tokens import DocBin
from spacy.lang.es.stop_words import STOP_WORDS
from tqdm import tqdm
import json
import os
import string
from unidecode import unidecode
import random
import time
random.seed(42)

print(os.getcwd())
# spacy.__version__
# Check GPU information
# !nvidia-smi

C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner


## Carga Datos

In [ ]:
dbs_path = 'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner\dbs'

In [2]:
path = r'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner'
df = pd.read_pickle(os.path.join(path,'df_nuevo.pkl'))

In [7]:
# Load the spaCy model for Spanish
nlp = spacy.load('es_core_news_sm')

def preprocess(text):
    # Step 1: Lowercasing
    text = text.lower()
    text = unidecode(text)
    
    # Step 2: Tokenization
    doc = nlp(text)

    # Step 3: Stopword removal
    # tokens = [token.text for token in doc if token.text not in STOP_WORDS]
    #tokens = [token.text for token in doc if token.text not in STOP_WORDS and token.text not in string.punctuation]
    
    # Step 4: Lemmatization
    #lemmatized_tokens = [token.lemma_ for token in doc if token.text not in STOP_WORDS]
    lemmatized_tokens = [token.lemma_ for token in doc if token.text not in STOP_WORDS and token.text not in string.punctuation]


    return " ".join(lemmatized_tokens)

# Example usage:
text_example = "Este es un ejemplo de ,texto en español que queremos procesar."
preprocessed_tokens = preprocess(text_example)

print("Original Text:", text_example)
print("Preprocessed Tokens:", preprocessed_tokens)

Original Text: Este es un ejemplo de ,texto en español que queremos procesar.
Preprocessed Tokens: ejemplo texto espanol procesar


In [3]:
len(df)

496930

## Prepro

### Dates

In [91]:
def get_date(file_name):
    return tuple([int(l) for l in file_name[3:14].split('/')]) 

In [97]:
%timeit
date_ls  = [*map(get_date,df['escrito'].values)]

In [98]:
# Create new columns in the existing DataFrame
df['dia'], df['mes'], df['año'] = zip(*date_ls)

### clasificamos

In [112]:
model_path = r'C:\Users\juani\Documents\3_My_Jupiter_Notebooks\0_github\ner\textcat_model_1'
textcat_model = spacy.load(os.path.join(model_path,'model-best'))

In [113]:
def get_proba(text=str):
    doc = textcat_model(text)
    return doc.cats['POS']


In [ ]:
test= [*map(get_proba,df[(df.año>2022)&(df.mes==6)].texto.values.tolist())]

In [119]:
test= [*map(get_proba,df[(df.año>2022)&(df.mes==1)].texto.values.tolist())]

In [148]:
df[(df.año>2022)&(df.mes==1)].texto.values.tolist()[200]

'\n\n\n\n\n\n \nSOCDUS S.A.\nCUIT 30-70757147-7 Esc. 1, 02/01/2023, Fº 2, Reg. 101 CABA. Protocolización Actas de Asamblea y Directorio \nambas del 14/03/2022. Designación Presidente: Inés Ofelia García. Director Suplente: Rolando Ariel Gutesman, \nambos con domicilio especial en Av. Belgrano 1683/87, Piso 3, CABA. Duración 3 ejercicios. Autorizado según \ninstrumento público Esc. Nº\xa01 de fecha 02/01/2023 Reg. Nº\xa0101\nPaula Andrea Di Salvo - Matrícula: 4857 C.E.C.B.A.\n'

In [153]:
start_time = time.time()

class_ls= [*map(get_proba,df.texto.values.tolist())]

print(f"Execution time: {time.time()- start_time} seconds")

Execution time: 31257.749974489212 seconds


In [158]:
class_ls[0], len(class_ls)

(0.9999781847000122, 496930)

In [160]:
df['class_prob'] = class_ls

In [162]:
print(path)

C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner


In [163]:
os.getcwd()

'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\0_github\\ner'

In [164]:
df.columns

Index(['chapter', 'escrito', 'texto', 'file', 'dia', 'mes', 'año',
       'class_prob'],
      dtype='object')

In [166]:
df.to_pickle(os.path.join(path,'df_nuevo.pkl'))

## Importamos Librerías

In [159]:
df.head()

,chapter,escrito,texto,file,dia,mes,año
0,CONVOCATORIAS Y AVISOS COMERCIALES,e. 01/06/2017 N° 37101/17 v. 01/06/2017,BAZAR MEGA S.A.\nConstitución: Por escritura 7...,seccion_segunda_20170601.pdf,1,6,2017
1,CONVOCATORIAS Y AVISOS COMERCIALES,e. 01/06/2017 N° 37342/17 v. 01/06/2017,\nBETHEL WORLD S.A.\nPor escritura 155 Folio 4...,seccion_segunda_20170601.pdf,1,6,2017
2,CONVOCATORIAS Y AVISOS COMERCIALES,e. 01/06/2017 N° 37349/17 v. 01/06/2017,\nBIGPAL S.A.\n1) 24/5/17; 2) Sergio Francisco...,seccion_segunda_20170601.pdf,1,6,2017
3,CONVOCATORIAS Y AVISOS COMERCIALES,e. 01/06/2017 N° 37374/17 v. 01/06/2017,\nBLUE FIVE S.A.\nEsc. 100 del 17/5/17: ROMERO...,seccion_segunda_20170601.pdf,1,6,2017
4,CONVOCATORIAS Y AVISOS COMERCIALES,e. 01/06/2017 N° 37325/17 v. 01/06/2017,\nCAMUZZI ARGENTINA S.A.\nSe comunica que por ...,seccion_segunda_20170601.pdf,1,6,2017


In [156]:
31257.749974489212 /60/60

8.682708326247003

In [151]:
for y in sorted(df.año.unique()):
    print(f'{y}: {len(df[(df.año==y)])}') 

2016: 1
2017: 45632
2018: 73019
2019: 73336
2020: 47896
2021: 13043
2022: 237629
2023: 6374


---